In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import networkx as nx
from dowhy import CausalModel
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import ElasticNetCV  
from sklearn.linear_model import LassoCV   

import warnings
warnings.filterwarnings('ignore')

## Read data

In [ ]:
df_daily = pd.read_csv('Processed data/valid_data_fe.csv')
df_daily.head()

## DiD model

In [3]:
# Define dependent and independent variable matrices
X = df_daily.drop(columns=['LCLid', 'Date', 'kwh_hh', 'Acorn', 'got_high_signal', 'got_low_signal', 'high_signal_intensity', 'low_signal_intensity', 'is_weekend'])
X = X.drop([col for col in X.columns if col.startswith('month')], axis=1) # Drop months
X = sm.add_constant(X)
y = df_daily['kwh_hh']

In [ ]:
# Fit model
model = sm.OLS(y, X)
results = model.fit(cov_type='HC1')
print(results.summary())

## DoWhy + EconML model

In [5]:
# Initialize directed graph
G = nx.DiGraph()

# List of causes
causes = ['day_of_week_Monday',
       'day_of_week_Saturday', 'day_of_week_Sunday', 'day_of_week_Thursday',
       'day_of_week_Tuesday', 'day_of_week_Wednesday', 'temp',
       'feelslike', 'precip', 'cloudcover', 'Acorn_enc'] 
       
# Add edges from each cause to 'kwh_hh'
for cause in causes:
    G.add_edge(cause, 'kwh_hh')

# Add the path from treat to treat_2013 to kwh_hh 
# Actual treatment variable is treat_2013, but treated households were already different in 2012
G.add_edges_from([
    ('treat', 'treat_2013'),
    ('treat_2013', 'kwh_hh'),
])

G.add_edges_from([
    ('treat', 'kwh_hh'),
])

# We have reasons to believe that treatment assignment may have depended on accorn status
G.add_edges_from([
    ('Acorn_enc', 'treat')
])

In [ ]:
# Draw the graph
plt.figure(figsize=(12, 8))

# Use shell layout for a more circular, cleaner arrangement
pos = nx.circular_layout(G) 

# Draw nodes with a color gradient and different sizes based on their degree
node_sizes = [3000 if node == 'kwh_hh' else 2000 for node in G.nodes()]
node_colors = ['lightgreen' if node == 'kwh_hh' else 'skyblue' for node in G.nodes()]
nx.draw_networkx_nodes(G, pos, node_size=node_sizes, node_color=node_colors, edgecolors='black')

# Draw edges with arrows and custom width/color
nx.draw_networkx_edges(G, pos, arrowstyle='-|>', arrowsize=45, edge_color='gray', width=2)

# Add labels to the nodes
nx.draw_networkx_labels(G, pos, font_size=12, font_color='black', font_weight='bold')

# Remove axis for a cleaner look
plt.axis('off')

# Show the graph
plt.tight_layout()
plt.savefig('Images/whole_network.png')
plt.show()

In [7]:
model = CausalModel(
   data=df_daily, 
   treatment=['treat_2013'],  # Actual treatment, whose effect we want to quantify
   common_causes = ['treat'], # Treated households were already different in 2012
   effect_modifiers=causes,
   outcome="kwh_hh",
   graph="\n".join(nx.generate_gml(G))
)

In [ ]:
model.view_model()

In [ ]:
identified_estimand = model.identify_effect(proceed_when_unidentifiable=True)
print(identified_estimand)

In [ ]:
dml_estimate = model.estimate_effect(identified_estimand, 
                                    method_name="backdoor.econml.dml.DML",
                                    control_value = 0,
                                    treatment_value = 1,
                                    #  target_units = lambda df_daily: df_daily["Acorn_enc"]>=15,  # condition used for CATE
                                    confidence_intervals=False,
                                    method_params={"init_params":{'model_y': GradientBoostingRegressor(random_state=42),
                                                                'model_t': GradientBoostingRegressor(random_state=42),
                                                                'featurizer':PolynomialFeatures(degree=2, include_bias=False),
                                                                'model_final': LassoCV(random_state=42),
                                                                'random_state': 42},
                                                    "fit_params":{}}
                                    )
print(dml_estimate)

In [ ]:
dml_estimate.value

#### Refute estimate
WARNING - High memory use. These three cells can take a lot of time to run.


In [ ]:
# Adding a random common cause variable
res_random=model.refute_estimate(identified_estimand, dml_estimate, method_name="random_common_cause", show_progress_bar=True)
print(res_random)

In [69]:
# Replacing treatment with a random (placebo) variable
res_placebo=model.refute_estimate(identified_estimand, dml_estimate,
        method_name="placebo_treatment_refuter", show_progress_bar=True, placebo_type="permute")
print(res_placebo)

In [ ]:
# Removing random subset of the data
res_subset=model.refute_estimate(identified_estimand, dml_estimate,
        method_name="data_subset_refuter", show_progress_bar=True, subset_fraction=0.9)
print(res_subset)

## Separate models for each type of price signal

In [13]:
# Initialize new directed graph
G_alt = nx.DiGraph()

# List of causes
causes = ['day_of_week_Monday',
       'day_of_week_Saturday', 'day_of_week_Sunday', 'day_of_week_Thursday',
       'day_of_week_Tuesday', 'day_of_week_Wednesday', 'temp',
       'feelslike', 'precip', 'cloudcover', 'Acorn_enc'] 
       
# Add edges from each cause to 'kwh_hh'
for cause in causes:
    G_alt.add_edge(cause, 'kwh_hh')

# Add the paths described in the question
# Path 1: treat to treatment in 2013 to getting high signal to consumption
G_alt.add_edges_from([
    ('treat', 'treat_2013'),
    ('treat_2013', 'got_high_signal'),
    ('got_high_signal', 'kwh_hh')
])

# Path 2: treat to treatment in 2013 to getting low signal to consumption
G_alt.add_edges_from([
    ('treat', 'treat_2013'),
    ('treat_2013', 'got_low_signal'),
    ('got_low_signal', 'kwh_hh')
])

# Even though our treatment here will be the type of signal received (if any), 
# knowing to be part of treatment group may have an impact on energy consumption
G_alt.add_edges_from([
    ('treat', 'treat_2013'),
    ('treat_2013', 'kwh_hh'),
])

# Again, treatment group was already different before 2013
G_alt.add_edges_from([
    ('treat', 'kwh_hh'),
])

# Again, it doesn't look like treatment was fully randomised
G_alt.add_edges_from([
    ('Acorn_enc', 'treat')
])

In [ ]:
# Draw the graph
plt.figure(figsize=(12, 8))

# Use shell layout for a more circular, cleaner arrangement
pos = nx.circular_layout(G_alt) 

# Draw nodes with a color gradient and different sizes based on their degree
node_sizes = [3000 if node == 'kwh_hh' else 2000 for node in G_alt.nodes()]
node_colors = ['lightgreen' if node == 'kwh_hh' else 'skyblue' for node in G_alt.nodes()]
nx.draw_networkx_nodes(G_alt, pos, node_size=node_sizes, node_color=node_colors, edgecolors='black')

# Draw edges with arrows and custom width/color
nx.draw_networkx_edges(G_alt, pos, arrowstyle='-|>', arrowsize=45, edge_color='gray', width=2)

# Add labels to the nodes
nx.draw_networkx_labels(G_alt, pos, font_size=12, font_color='black', font_weight='bold')

# Remove axis for a cleaner look
plt.axis('off')

# Show the graph
plt.tight_layout()
plt.savefig('Images/whole_network_signals.png')
plt.show()



In [ ]:
# Low price signal model
# Step 1. Define model
model_low = CausalModel(
   data=df_daily, # some pandas dataframe
   treatment=['got_low_signal'],
   common_causes = ['treat', 'treat_2013'],
   effect_modifiers=causes,
   outcome="kwh_hh",
   graph="\n".join(nx.generate_gml(G_alt))
)

# Step 2. Identify estimand
identified_estimand_low = model_low.identify_effect(proceed_when_unidentifiable=True)
print(identified_estimand_low)

# Step 3. Quantify causal estimate
causal_estimate_low = model_low.estimate_effect(identified_estimand_low, 
                                    method_name="backdoor.econml.dml.DML",
                                    control_value = 0,
                                    treatment_value = 1,
                                    confidence_intervals=False,
                                    method_params={"init_params":{'model_y': GradientBoostingRegressor(random_state=42),
                                                                'model_t': GradientBoostingRegressor(random_state=42),
                                                                'featurizer':PolynomialFeatures(degree=2, include_bias=False),
                                                                'model_final': ElasticNetCV(random_state=42),
                                                                'random_state': 42},
                                                    "fit_params":{}}
                                    )
print('Causal estimate value:', round(causal_estimate_low.value, 4))

In [ ]:
# High price signal model
# Step 1. Define model
model_high = CausalModel(
   data=df_daily, # some pandas dataframe
   treatment=['got_high_signal'],
   common_causes = ['treat', 'treat_2013'],
   effect_modifiers=causes.append('Acorn_enc'),
   outcome="kwh_hh",
   graph="\n".join(nx.generate_gml(G_alt))
)

# Step 2. Identify estimand
identified_estimand_high = model_high.identify_effect(proceed_when_unidentifiable=True)
print(identified_estimand_high)

# Step 3. Quantify causal estimate
causal_estimate_high = model_high.estimate_effect(identified_estimand_high, 
                                    method_name="backdoor.econml.dml.DML",
                                    control_value = 0,
                                    treatment_value = 1,
                                    confidence_intervals=False,
                                    method_params={"init_params":{'model_y': GradientBoostingRegressor(random_state=42),
                                                                'model_t': GradientBoostingRegressor(random_state=42),
                                                                'featurizer':PolynomialFeatures(degree=2, include_bias=False),
                                                                'model_final': ElasticNetCV(random_state=42),
                                                                'random_state': 42},
                                                    "fit_params":{}}
                                    )
print('Causal estimate value:', round(causal_estimate_high.value, 4))
# print('Causal estimate value:', causal_estimate_high.get_confidence_intervals())

In [ ]:
model_high.view_model()

## EXTRA - Calculating consumptions

In [ ]:
treatment_effect = 0.2565

# Keep only 2013
df_days = df_daily.copy()
df_days['Date'] = pd.to_datetime(df_days['Date'])
df_days = df_days[df_days['Date'].dt.year == 2013]

# Add "saved energy" to treated households
df_days['kwh_hh_alt'] = np.where(df_days['treat'] == 1, df_days['kwh_hh'] + treatment_effect, df_days['kwh_hh'])

In [ ]:
# Consumption totals 
actual_cons = df_days['kwh_hh'][df_days['treat']==1].sum()          # Actual consumption
potential_cons = df_days['kwh_hh_alt'][df_days['treat']==1].sum()   # Counterfactual consumption - what would have been consumed where it not for the DToU tariff
diff_cons = actual_cons - potential_cons                            # Difference between actual and counterfactual consumption

print('Actual consumption:', actual_cons)
print('Potential consumption:', potential_cons)

In [ ]:
# Per household totals
print('Average expenditure difference:', diff_cons * 14.228 / 100 / 77)
print('Average energy difference:', diff_cons / 77)

In [ ]:
# Average household consumption in 2013
total_cons = df_days.groupby(['LCLid'])['kwh_hh'].sum()
total_cons.mean()